In [11]:
import numpy as np
from datetime import time

In [14]:
%%time
table1 = np.genfromtxt("data/household_power_consumption.txt", delimiter=';', usecols=range(2, 9), skip_header=True)
table2 = np.loadtxt("data/household_power_consumption.txt", delimiter=';', dtype=str, usecols=range(0,2), skiprows=1)

Wall time: 1min 6s


In [52]:
%%time

def split_datetime(row):
    result = []
    for item in row[0].split('/'):
        result.append(item)
    for item in row[1].split(':')[:2]:
        result.append(item)
    return result

temp = np.apply_along_axis(split_datetime, 1, table2)
table = np.concatenate((temp, table1), axis=1).astype('float32')

Wall time: 1min 9s


In [54]:
table

array([[1.600e+01, 1.200e+01, 2.006e+03, ..., 0.000e+00, 1.000e+00,
        1.700e+01],
       [1.600e+01, 1.200e+01, 2.006e+03, ..., 0.000e+00, 1.000e+00,
        1.600e+01],
       [1.600e+01, 1.200e+01, 2.006e+03, ..., 0.000e+00, 2.000e+00,
        1.700e+01],
       ...,
       [2.600e+01, 1.100e+01, 2.010e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.600e+01, 1.100e+01, 2.010e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.600e+01, 1.100e+01, 2.010e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

In [55]:
np.set_printoptions(suppress=True, precision=3)
table

array([[  16.,   12., 2006., ...,    0.,    1.,   17.],
       [  16.,   12., 2006., ...,    0.,    1.,   16.],
       [  16.,   12., 2006., ...,    0.,    2.,   17.],
       ...,
       [  26.,   11., 2010., ...,    0.,    0.,    0.],
       [  26.,   11., 2010., ...,    0.,    0.,    0.],
       [  26.,   11., 2010., ...,    0.,    0.,    0.]], dtype=float32)

In [58]:
%%time
table = table[~np.any(np.isnan(table), axis=1),:]

Wall time: 335 ms


## (1)
Обрати всі рядки, у яких загальна актив. споживана потужн. > 5 кВт

In [ ]:
%%time
table[table[:,5] > 5,:]

## (2)
Обрати всі рядки, у яких вольтаж > 235 В.

In [63]:
%%time
table[table[:,7] > 235,:]

Wall time: 159 ms


array([[  16.,   12., 2006., ...,    0.,    1.,   17.],
       [  16.,   12., 2006., ...,    0.,    2.,   17.],
       [  16.,   12., 2006., ...,    0.,    1.,   17.],
       ...,
       [  26.,   11., 2010., ...,    0.,    0.,    0.],
       [  26.,   11., 2010., ...,    0.,    0.,    0.],
       [  26.,   11., 2010., ...,    0.,    0.,    0.]], dtype=float32)

## (3)
Обрати всі рядки, у яких 19<I<20 A, для них виявити ті, у яких пральна машина та холодильних
споживають більше, ніж бойлер та кондиціонер

In [103]:
%%time
table[ (table[:,8] >= 19) &  (table[:,8] <= 20) & (table[:,10] > table[:,11]), :]

Wall time: 66.9 ms


array([[  16.,   12., 2006., ...,    0.,   37.,   16.],
       [  17.,   12., 2006., ...,    0.,   13.,    0.],
       [  17.,   12., 2006., ...,    0.,   27.,    0.],
       ...,
       [  24.,   11., 2010., ...,    0.,   40.,   17.],
       [  24.,   11., 2010., ...,    0.,   39.,   17.],
       [  24.,   11., 2010., ...,    0.,   39.,   17.]], dtype=float32)

## (4)
Обрати випадковим чином 500000 рядків (без повторів
елементів вибірки), для них обчислити середні величини усіх 3-х
груп споживання електричної енергії, а також

In [87]:
%%time
rand = np.random.choice(np.arange(0,len(table)), 500000, replace=False)
table[rand,:]

Wall time: 244 ms


array([[  28.,    5., 2009., ...,    0.,    0.,    0.],
       [  25.,    7., 2008., ...,    0.,    0.,    0.],
       [   4.,    1., 2009., ...,    0.,    0.,    0.],
       ...,
       [  14.,    3., 2007., ...,    0.,    1.,    0.],
       [  29.,    7., 2010., ...,    0.,    0.,   18.],
       [  16.,    6., 2009., ...,    1.,    0.,   18.]], dtype=float32)

In [86]:
%%time
Mean1 = np.mean(table[:,9])
Mean2 = np.mean(table[:,10])
Mean3 = np.mean(table[:,11])
print(f"Mean1: {Mean1}",f"Mean2: {Mean2}", f"Mean3: {Mean3}")

Mean1: 1.1219233274459839 Mean2: 1.2985199689865112 Mean3: 6.458447456359863
Wall time: 35.7 ms


## (5)
Обрати ті рядки, які після 18-00 споживають > 6
кВт за хвилину в середньому, серед відібраних визначити ті, у яких
основне споживання електроенергії у вказаний проміжок часу
припадає на пральну машину, сушарку, холодильник та освітлення
(група 2 є найбільшою), а потім обрати кожен третій результат із
першої половини та кожен четвертий результат із другої половини.

In [104]:
%%time
condition = table[(table[:,3] >= 18) & (table[:,5] > 6)]
max_met2 = condition[(condition[:,10] > condition[:,9]) & (condition[:,10] >= condition[:,11]), :]
first, second = np.array_split(max_met2, 2)

Wall time: 26.9 ms


In [105]:
first[::3]

array([[  16.,   12., 2006., ...,    0.,   37.,   17.],
       [  16.,   12., 2006., ...,    0.,   36.,   17.],
       [  28.,   12., 2006., ...,    1.,   36.,   17.],
       ...,
       [  23.,    3., 2008., ...,    0.,   72.,   17.],
       [  23.,    3., 2008., ...,    0.,   73.,   16.],
       [  20.,    4., 2008., ...,    1.,   72.,   17.]], dtype=float32)

In [106]:
second[::4]

array([[  20.,    4., 2008., ...,    1.,   71.,   17.],
       [  20.,    4., 2008., ...,    1.,   68.,   17.],
       [  20.,    4., 2008., ...,    1.,   37.,   17.],
       ...,
       [  20.,   11., 2010., ...,   15.,   34.,   17.],
       [  20.,   11., 2010., ...,   14.,   35.,   16.],
       [  20.,   11., 2010., ...,   13.,   39.,   16.]], dtype=float32)